# Coursework Part 1: Detecting Spam with Spark

These are the tasks for IN432 Big Data coursework 2018, part 1.  

This coursework is about classification of e-mail messages as spam or non-spam in Spark. We will go through the whole process from loading preprocessing to training and testing classifiers in a distributed way in Spark. We wil use the techniques shown in the lextures and labs. I will also introduce here a few additional elements, such as the NLTK and some of the preprocessing and machine learning functions that come with Spark. You are not expected to need anything beyond the material handed out so far and in some cases the Spark documentation, to which I have put links in this document.  

The structure is similar to the lab sheets. I provide a code structure with gaps that you are supposed to file. In addition you should run 2 small experiments and comment on the results. The lines where you are supposed to add code or take another action are marked with ">>>" 
please leave the ">>>" in the text, comment out that line, and write your own code in the next line using a copy of that line as a starting point.

I have added numerous comments in text cells and the code cells to guid you through the program. Please read them carefully and ask if anything is unclear. 

Once you have completed the tasks, don't delete the outpus, but downlaod the notebook (outputs will be included) and upload it into the coursework submission area on Moodle. The coursework part counts for 25% or the total coursework.

## Load and prepare the data

We will use the lingspam dataset in this coursework (see [http://csmining.org/index.php/ling-spam-datasets.html](http://csmining.org/index.php/ling-spam-datasets.html) for more information).

The next cell is only needed if you haven't cloned the repository in week 2 or later (but it doesn't do harm to run it). 

In [1]:
%cd ~/notebook/work/
!git clone https://github.com/tweyde/City-Data-Science.git

/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb1c-2a2fc727357cc3-3b10766cfe3e/notebook/work
fatal: destination path 'City-Data-Science' already exists and is not an empty directory.


In [1]:
%cd ~/notebook/work/City-Data-Science/
!git pull
%cd ./datasets/ 
#we need to use a magic command (starting with '%') here.
print(">>> Extracting the ling_spam dataset, this can take a moment.")
!tar -xf lingspam_public02.tar.gz
# '!' calls a program on the machine (the DSX service runs on Linux machines).
print(">>> Unzipping finished.")
# We now have a new dataset in directory 'bare'.
%cd lingspam_public/bare 
print(">>> pwd ")
!pwd
print(">>> ls ")
!ls
# the line before last of output should show "part1 part10 part2  part3  part4  part5  part6  part7 part8 part9"
%cd ..

/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb1c-2a2fc727357cc3-3b10766cfe3e/notebook/work/City-Data-Science
Already up-to-date.
/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb1c-2a2fc727357cc3-3b10766cfe3e/notebook/work/City-Data-Science/datasets
>>> Extracting the ling_spam dataset, this can take a moment.
>>> Unzipping finished.
/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb1c-2a2fc727357cc3-3b10766cfe3e/notebook/work/City-Data-Science/datasets/lingspam_public/bare
>>> pwd 
/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb1c-2a2fc727357cc3-3b10766cfe3e/notebook/work/City-Data-Science/datasets/lingspam_public/bare
>>> ls 
part1  part10  part2  part3  part4  part5  part6  part7  part8	part9
/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb1c-2a2fc727357cc3-3b10766cfe3e/notebook/work/City-Data-Science/datasets/lingspam_public


### Tools for Troubleshooting

Normally, DSX works reliably, but there are two issues that have occured. We have solutions for them that you can use with the following cells. 
If other problems occur, reloading the page and/or restarting the Kernel can help. 

In [2]:
# try this in case of "sc undefined" or similar errors, should normally not be necessary.
from pyspark import SparkContext
sc = spark.sparkContext

In [26]:
# sometimes, when you have multiple notebooks open at the same time, you might get an error that the metastore_db is not accessible.
# We can not prevent this form happening on DSX (apart from not opening more than one notebook at a time).
# If it does happen you need to delete the metastore_db. The path of the metastore_db is in the error messages and it's typically 
# lond like this example: 
# /gpfs/global_fs01/sym_shared/YPProdSpark/user/s832-dfe96c6e1f1d61-70d619a53771/notebook/jupyter-rt/kernel-cdcf5f73-9afb-481d-ac40-a210a649eb69-20180222_154448/metastore_db
# once you have it, you can use it with !rm -Rf to delete it:
# !rm -Rf <Put the path of the metastore_db here>
# !rm -Rf /gpfs/global_fs01/sym_shared/YPProdSpark/user/sb1c-2a2fc727357cc3-3b10766cfe3e/notebook/jupyter-rt/kernel-d80a2973-6a14-4764-b9ad-036f0bad1461-20180228_215813/metastore_db
!rm -Rf /gpfs/global_fs01/sym_shared/YPProdSpark/user/sb1c-2a2fc727357cc3-3b10766cfe3e/notebook/jupyter-rt/kernel-246d0ae7-6f6d-4d0c-9698-0feda050837b-20180309_152415/metastore_db


## Task 1) Read the dataset and create RDDs 
a) Start by reading the directory with text files from the file system (`~/notebook/work/City-Data-Science/datasets/bare`). Load all text files per dirctory (part1,part2, ... ,part10) using `wholeTextFiles()`, which creates one RDD per part, containing tuples (filename,text). This is a good choice as the text files are small. (5%)

b) We will use one of the RDDs as test set, the rest as training set. For the training set you need to create the union of the remaining RDDs. (5%)

b) Remove the path and extension from the filename using the regular expression provided (5%).

If the filename starts with 'spmsg' it is spam, otherwise it is not. We'll use that later to train a classifier. 

We will put the code in each cell into a function that we can reuse later. In this way we can develop the whole preprocessing with the smaller test set and apply it to the training set once we know that everything works. 

In [53]:
from pathlib import Path
import re

def makeTestTrainRDDs(pathString):
    """ Takes one of the four subdirectories of the lingspam dataset and returns two RDDs one each for testing and training. """
    
    # We should see10 parts that we can use for creating train and test sets.
    p = Path(pathString) # gets a path object representing the current directory path.
    dirs = list(p.iterdir()) # get the directories part1 ... part10. 
    # print(dirs) # Print to check that you have the right directory. You can comment this out when checked. 
    
    rddList = [] # create a list for the RDDs
    # now create an RDD for each 'part' directory and add them to rddList
    for d in dirs: # iterate through the directories
        # >>>     rdd = ... #>>> # read the files in the directory 
        rdd = sc.wholeTextFiles(str(d.absolute())) #>>> # read the files in the directory (need to stringify and calling absolute method ensures correct directory is called irrespective of current location)
        # >>>     ... #>>> append the RDD to the rddList
        rddList.append(rdd) # append the RDD to the rddList
        
        
    # print('len(rddList)',len(rddList))  # we should now have 10 RDDs in the list # just for testing
    # print(rddList[1].take(1)) # just for testing, comment out when it works.
    
    testRDD1 = rddList[9] # set the test set
    trainRDD1 = rddList[0] # start the training set from 0 and 
    # now loop over the range from 1 to 9(exclusive) to create a union of the remaining RDDs
    for i in range(1,9):
        trainRDD1 = trainRDD1.union(rddList[i]) #>>> create a union of the current and the next 
            # RDD in the list, so that in the end we have a union of all parts 0-8. (9 ist used as test set)
            
    # both RDDs should remove the paths and extensions from the filename. 
    #>>> This regular expression will do it: re.split('[/\.]', fn_txt[0])[-2]
    #>>> apply it to the filenames in train and test RDD with a lambda
    testRDD2 = testRDD1.map(lambda fn_txt: (re.split('[/\.]', fn_txt[0])[-2], fn_txt[1])) # split by forward slash or point character and extracting just the file name without the extension
    trainRDD2 = trainRDD1.map(lambda fn_txt: (re.split('[/\.]', fn_txt[0])[-2], fn_txt[1])) # split by forward slash or point character and extracting just the file name without the extension
    return (trainRDD2,testRDD2)

# this makes sure we are in the right directory
%cd ~/notebook/work/City-Data-Science/datasets/lingspam_public/
# this should show "bare  lemm  lemm_stop  readme.txt  stop"
!ls

# the code below is for testing the function makeTestTrainRDDs
trainRDD_testRDD = makeTestTrainRDDs('bare') # read from the 'bare' directory - this takes a bit of time
(trainRDD,testRDD) = trainRDD_testRDD # unpack the returned tuple
print('created the RDDs') # notify the user, so that we can figure out where things went wrong if they do.
print('testRDD.count(): ',testRDD.count()) # should be 291 
#print('trainRDD.count(): ',trainRDD.count()) # should be 2602 - commented out to save time
print('testRDD.getNumPartitions()',testRDD.getNumPartitions()) # normally 2 on DSX
print('testRDD.getStorageLevel()',testRDD.getStorageLevel()) # Serialized 1x Replicated on DSX
print('testRDD.take(1): ',testRDD.take(1)) # should be (filename,[tokens]) 
rdd1 = testRDD # use this for developemnt in the next tasks 

/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb1c-2a2fc727357cc3-3b10766cfe3e/notebook/work/City-Data-Science/datasets/lingspam_public
bare  lemm  lemm_stop  readme.txt  stop
created the RDDs
testRDD.count():  291
testRDD.getNumPartitions() 2
testRDD.getStorageLevel() Serialized 1x Replicated
testRDD.take(1):  [('9-66msg1', "Subject: xth conference of nordic and general ling .\n\nthe tenth conference of nordic and general linguistics will be held in reykjavik , iceland , from saturday june 6 , to monday june 8 , 1998 . it is organized by the institute of linguistics , university of iceland . the deadline for pre-registration at a reduced price is january 31 , 1998 . pre - registration forms and further information can be found on our web site ( http : / / www . rhi . hi . is / ~ nordconf ) and can also be mailed or e-mailed upon request . papers on any linguistic topic are invited , especially papers on synchronic and diachronic aspects of the nordic languages . invited speakers : and

## Task 2) Tokenize and remove punctuation

Now we need to split the words, a process called *tokenization* by linguists, and remove punctuation. 

We will use the Python [Natural Language Toolkit](http://www.nltk.org) *NLTK* to do the tokenization (rather than splitting ourselves, as these specialist tools usually do that we can ourselves). We use the NLTK function word_tokenize, see here for a code example: [http://www.nltk.org/book/ch03.html](http://www.nltk.org/book/ch03.html). 5%

Then we will remove punctuation. There is no specific funtion for this, so we use a regular expression (see here for info [https://docs.python.org/3/library/re.html?highlight=re#module-re](https://docs.python.org/3/library/re.html?highlight=re#module-re)) in a list comprehension (here's a nice visual explanation: [http://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/](http://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/)). 5% 

We use a new technique here: we separate keys and values of the RDD, using the RDD functions `keys()` and `values()`, which yield each a new RDD. Then we process the values and *zip* them together with the keys again. See here for documentation: [http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.zip](http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.zip).  We wrap the whole sequence into one function `prepareTokenRDD` for later use. 5%

In [54]:
import nltk
import re
from nltk.corpus import stopwords

def tokenize(text):
    """ Apply the nltk.word_tokenize() method to our text, return the token list. """
    nltk.download('punkt') # this loads the standard NLTK tokenizer model 
    # it is important that this is done here in the function, as it needs to be done on every worker.
    # If we do the download outside a this function, it would only be executed on the driver     
    # >>>    return ... # use the nltk function word_tokenize
    from nltk import word_tokenize # Can remove as import is not required within function
    tokens = word_tokenize(text)
    return tokens # use the nltk function word_tokenize
    
def removePunctuation(tokens):
    """ Remove punctuation characters from all tokens in a provided list. """
    # this will remove all punctiation from string s: re.sub('[()\[\],.?!";_]','',s)
    # tokens2 =  [...] # use a list comprehension to remove punctuaton
    tokens2 =  [re.sub('[()\[\],.?!";_]','',s) for s in tokens] # use a list comprehension to remove punctuaton
    return tokens2
    
def prepareTokenRDD(fn_txt_RDD):
    """ Take an RDD with (filename,text) elements and transform it into a (filename,[token ...]) RDD without punctuation characters. """
    rdd_vals2 = fn_txt_RDD.values() # It's convenient to process only the values. 
    rdd_vals3 = rdd_vals2.map(tokenize) # Create a tokenised version of the values by mapping
    rdd_vals4 = rdd_vals3.map(removePunctuation) # remove punctuation from the values
    rdd4 = fn_txt_RDD.keys().zip(rdd_vals4) # we zip the two RDDs together 
    # i.e. produce tuples with one item from each RDD.
    # This works because we have only applied mappings to the values, 
    # therefore the items in both RDDs are still aligned.
    # now remove any empty value strings (i.e. length 0) that we may have created by removing punctiation.
# >>>    rdd5 = rdd4 ... # do the removal using RDD.filter and a lambda. TIP len(s) gives you the lenght of string. 
    rdd5 = rdd4.map(lambda f_t: (f_t[0], list(filter(lambda s: len(s) > 0, f_t[1])))) # do the removal using RDD.filter and a lambda. TIP len(s) gives you the lenght of string.
    rdd6 = rdd5.filter(lambda f_t: len(f_t[1]) > 0) # remove items without tokens using RDD.filter and a lambda. 
#     # >>> Question: why should this be done after zipping the keys and values together?
#     Removing empty tokens before zipping would mean that the key and value RDDs would no longer align and therefore you cannot zip together the two RDD objects.
    return rdd6 
rdd2 = prepareTokenRDD(rdd1) # Use the test set for now, because it is smaller
print(rdd2.take(1)) # For checking result of task 2. 

[('9-66msg1', ['Subject', ':', 'xth', 'conference', 'of', 'nordic', 'and', 'general', 'ling', 'the', 'tenth', 'conference', 'of', 'nordic', 'and', 'general', 'linguistics', 'will', 'be', 'held', 'in', 'reykjavik', 'iceland', 'from', 'saturday', 'june', '6', 'to', 'monday', 'june', '8', '1998', 'it', 'is', 'organized', 'by', 'the', 'institute', 'of', 'linguistics', 'university', 'of', 'iceland', 'the', 'deadline', 'for', 'pre-registration', 'at', 'a', 'reduced', 'price', 'is', 'january', '31', '1998', 'pre', '-', 'registration', 'forms', 'and', 'further', 'information', 'can', 'be', 'found', 'on', 'our', 'web', 'site', 'http', ':', '/', '/', 'www', 'rhi', 'hi', 'is', '/', '~', 'nordconf', 'and', 'can', 'also', 'be', 'mailed', 'or', 'e-mailed', 'upon', 'request', 'papers', 'on', 'any', 'linguistic', 'topic', 'are', 'invited', 'especially', 'papers', 'on', 'synchronic', 'and', 'diachronic', 'aspects', 'of', 'the', 'nordic', 'languages', 'invited', 'speakers', ':', 'anders', 'holmberg', 't

## Task 3) Creating normalised TF.IDF vectors of defined dimensionality, measure the effect of caching.

We use the hashing trick to create fixed size TF vectors directly from the word list now (slightly different from the previous lab, where we used *(word,count)* pairs.). Write a bit of code as needed. (5%)

Then we'll use the IDF and Normalizer functions provided by Spark. They use a slightly different pattern than RDD.map and reduce, have a look at the examples here in the documentation for Normalizer  and IDF:
[http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.Normalizer](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.Normalizer), [http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.IDF](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.IDF) (5%)

We want control of the dimensionality in the `normTFIDF` function, so we introduce an argument into our functions that enables us to vary dimensionalty later. Here is also an opportunity to benefit from caching, i.e. persisting the RDD after use, so that it will not be recomputed.  (5%)

In [55]:
# use the hashing trick to create a fixed-size vector from a word list
def hashing_vectorize(text,N): # arguments: the list and the size of the output vector
    v = [0] * N  # create vector of 0s
    for word in text: # iterate through the words 
        # >>>              # get the hash value 
        hash_value = hash(word)
        # >>>              # add 1 at the hashed address 
        v[hash_value % N] += 1
    return v # return hashed word vector

from pyspark.mllib.feature import IDF, Normalizer

def normTFIDF(fn_tokens_RDD, vecDim, caching=True):
    keysRDD = fn_tokens_RDD.keys()
    tokensRDD = fn_tokens_RDD.values()
    tfVecRDD = tokensRDD.map(lambda tokens: hashing_vectorize(tokens,vecDim)) #>>> passing the vecDim value. TIP: you need a lambda. # generates an RDD of hash vectors
    # print(tfVecRDD.take(3))
    
# For loop below look for TF vector with 0 values the corresponding position in the TF vector will contain non zero values when TF.IDF is calculated
#     count = 0
#     for v in tfVecRDD.collect():
#         if 0 in v:
#             print (v)
#         count += 1 # count the number of documents

    if caching:
        tfVecRDD.persist(StorageLevel.MEMORY_ONLY) # since we will read more than once, caching in Memory will make things quicker.
    
    idf = IDF() # create IDF object
    idfModel = idf.fit(tfVecRDD) # calculate IDF values
    tfIdfRDD = idfModel.transform(tfVecRDD) # 2nd pass needed (see lecture slides), transforms RDD
    #     Note that in the output shown below all terms represented by positions 1-10 in the TF vector were found in all documents except for position 7 hence the IDF value will work out to zero
    #     for all values of the TF.IDF vector (i.e. IDF= Log(No of docs/ No of docs containing term) = Log (291/291) = 0 (for the test data)) except for position 7 where one doc is missing hence for this 
    #     corresponding term IDF = Log(291/290) which equals 0.00344 and therefore when you multiply to the corresponding element in tfVecRDD vector you get the 
    #     non-zero values shown eg the for the tfVecRDD vector representing the second document we see the following [30, 27, 12, 26, 10, 21, 9, 19, 22, 20] where we see that the count for the seventh postion is 9 
    #     and therefore 9 * 0.00344 = 0.0309 which is the resultant value we obtain in the seventh position for the corresponding tfIdfRDD DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0309, 0.0, 0.0, 0.0]. 
    #     All other values in the tfIdfRDD vector are zero as they are multipled by the IDF term Log (291/291) which equals 0.
    # print (tfIdfRDD.take(3))
    
    # >>>    norm = ... # create a Normalizer object like in the example linked above
    norm = Normalizer()  # default is p=2 which ensures normalisation relative to the product of doc and query vector lengths
    # >>>    normTfIdfRDD = norm. ... # and apply it to the tfIdfRDD 
    normTfIdfRDD = norm.transform(tfIdfRDD)
    
    # print statement to check RDD structure and values
    # print("tfIdfRDD check", tfIdfRDD.take(5))
    # print (normTfIdfRDD.take(1))
    
    # >>>    zippedRDD = ... # zip the keys and values together
    zippedRDD = keysRDD.zip(normTfIdfRDD)
    return zippedRDD

testDim = 10 # too small for good accuracy, but OK for testing
rdd3 = normTFIDF(rdd2, testDim, True) # test our
print(rdd3.take(1)) # we should now have tuples with ('filename',[N-dim vector])

# e.g. [('9-1142msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.4097, 0.0, 0.0, 0.0, 0.9122, 0.0]))]

[('9-66msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]))]


### Task 3a) Caching experiment

The normTFIDF let's us switch caching on or off. Write a bit of code that measures the effect of caching by takes the time for both options. Use the time function as shown in lecture 3, slide 47. Remember that you need to call an action on an RDD to trigger full execution. 

Add a short comment on the result (why is there an effect, why of the size that it is?). Remember that this is wall clock time, i.e. you may get noisy results. (10%)

In [58]:
#run a small experiment with caching set to True or False, 3 times each

from time import time
import numpy as np

resCaching = [] # for storing results
resNoCache = [] # for storing results

for i in range(3): # 3 samples
    # >>>  # start timer
    startTime = time()
    
    testRDD1 = normTFIDF(rdd2, testDim, True) # 
    
    # >>>     # call an action on the RDD to force execution
    testRDD1_9_6 = testRDD1.filter(lambda f_v: f_v[0].startswith('9-')) # generate an RDD containing all the keys that start with 9-6
    print(testRDD1_9_6.take(3))
    
    # >>>  # end timer
    endTime = time()
    
    resCaching.append( endTime - startTime ) # calculate the difference
    
    # >>>  # start timer
    startTime = time()
    
    testRDD2 = normTFIDF(rdd2, testDim, False) 
    
    # >>>  # call an action to force execution   
    testRDD2_9_6 = testRDD2.filter(lambda f_v: f_v[0].startswith('9-'))
    print(testRDD2_9_6.take(3))
    
    # >>>  # end timer
    endTime = time()
    
    resNoCache.append( endTime - startTime )
    
# >>> meanTimeCaching = # calculate average times
meanTimeCaching = np.mean(resCaching)
# >>> meanTimeNoCache = # calculate average times
meanTimeNoCache = np.mean(resNoCache)

print('Creating TF.IDF vectors, 3 trials - mean time with caching: ', meanTimeCaching, ', mean time without caching: ', meanTimeNoCache)
# >>> # add your results and comments here 
#  Caching holds the data assigned to testRDD1 in this example in cache memory (a form of fast access memory embedded in the CPU which can be anywhere between 10-100 times faster than RAM and upwards of 1 million times faster than accessing data from disk) as opposed to the data assigned to testRDD2 which is held in distributed disk storage. 
# This means that whenever you call an action on testRDD2 the RDD needs to first be re-computed from distriubuted storage before the action can be executed. 
#  This process can significantly impact on processing time.
# However because testRDD1 is held in memory actions performed on this RDD can be executed immediately without having to perform any computation from distributed storage. 
# The true difference in processing time can not be demonstrated using wall clock times as the time reflected in this measurement are affected by other external factors such as other processes running on the machine at the same time. 
# However despite this noise we can see in this example that by caching we approximately reduce processing time by one third from 18.71 to 12.16 seconds.


[('9-66msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-751msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-628msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]))]
[('9-66msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-751msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-628msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]))]
[('9-66msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-751msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-628msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]))]
[('9-66msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-751msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-628msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]))]
[('9-66msg1', DenseVector([0.0, 0.0, 0.0

## Task 4) Create LabeledPoints 

Determine whether the file is spam (i.e. the filename contains ’spmsg’) and replace the filename by a 1 (spam) or 0 (non-spam) accordingly. Use `RDD.map()` to create an RDD of LabeledPoint objects. See here [http://spark.apache.org/docs/2.1.0/mllib-linear-methods.html#logistic-regression](http://spark.apache.org/docs/2.1.0/mllib-linear-methods.html#logistic-regression) for an example, and here [http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.regression.LabeledPoint](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.regression.LabeledPoint) for the `LabeledPoint` documentation. (10%)

There is a handy function of Python strings called startswith: e.g. 'abc'.startswith('ab) will return true. The relevant Python syntax here is a conditional expression: **``<a> if <yourCondition> else <b>``**, i.e. 1 if the filename starts with 'spmsg' and otherwise 0.

In [59]:
from pyspark.mllib.regression import LabeledPoint

# creatate labelled points of vector size N out of an RDD with normalised (filename [(word,count), ...]) items
def makeLabeledPoints(fn_vec_RDD): # RDD and N needed 
    # we determine the true class as encoded in the filename and represent as 1 (samp) or 0 (good)
    # >>>    cls_vec_RDD = fn_vec_RDD.map(lambda ... ) # use a conditional expression to get the class label (True or False)
    cls_vec_RDD = fn_vec_RDD.map(lambda f_v: (1, f_v[1])  if f_v[0].startswith('spmsg') else (0, f_v[1])) # use a conditional expression to get the class label (True or False)

    # now we can create the LabeledPoint objects with (class,vector) arguments
    lp_RDD = cls_vec_RDD.map(lambda cls_vec: LabeledPoint(cls_vec[0],cls_vec[1]) ) 
    return lp_RDD


# for testing
testLpRDD = makeLabeledPoints(rdd3) 
print(testLpRDD.take(1)) 
# should look similar to this: [LabeledPoint(0.0, [0.0,0.0,0.0,0.0,0.40968062880166006,0.0,0.0,0.0,0.9122290186048,0.0])]

[LabeledPoint(0.0, [0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0])]


## Task 5) Complete the preprocessing 

It will be useful to have a single function to do the preprocessing. So integrate everything here. (5%)

In [60]:
# now we can apply the preprocessing chain to the data loaded in task 1 
# N is for controlling the vector size
def preprocess(rawRDD,N):
    """ take a (filename,text) RDD and transform into LabelledPoint objects 
        with class labels and a TF.IDF vector with N dimensions. 
    """
    tokenRDD = prepareTokenRDD(rawRDD) # task 2
    tfIdfRDD = normTFIDF(tokenRDD,N) # task 3
    lpRDD = makeLabeledPoints(tfIdfRDD) # task 4
    return lpRDD # return RDD with LabeledPoints

# and with this we can start the whole process from a directory, N is again the vector size
def loadAndPreprocess(directory,N):
    """ load lingspam data from a directory and create a training and test set of preprocessed data """
    # >>>    trainRDD_testRDD = ... # read from the directory using the function created in task 1
    trainRDD_testRDD = makeTestTrainRDDs(directory) # read from the directory using the function created in task 1 
    # >>>     # unpack the returned tuple
    (trainRDD, testRDD) = trainRDD_testRDD # unpack the returned tuple
    return (preprocess(trainRDD,N),preprocess(testRDD,N)) # apply the preprocessing funcion defined above

trainLpRDD = preprocess(trainRDD,testDim) # prepare the training data
print(testLpRDD.take(1)) # should look similar to previous cell's output

train_test_LpRDD = loadAndPreprocess('lemm',100) # let's re-run with another vector size
(trainLpRDD,testLpRDD) = train_test_LpRDD 
print(testLpRDD.take(1))
print(trainLpRDD.take(1))

[LabeledPoint(0.0, [0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0])]
[LabeledPoint(0.0, [0.0173065598238,0.0450282043828,0.07637472436,0.0428900460529,0.0730278218162,0.13845247859,0.207468110614,0.0907447439736,0.099848432519,0.15975749203,0.0568299720098,0.175748800957,0.173650740836,0.0,0.0841990489641,0.0318533592129,0.0271491006488,0.022397244891,0.0288613186302,0.0650231550584,0.237523491514,0.0431219297594,0.0,0.134049457572,0.0,0.0831523206436,0.0985569335186,0.0236171609595,0.0658853314433,0.141869820965,0.138587201073,0.0767548415888,0.0817292485974,0.129408153043,0.160954127147,0.0,0.120059729193,0.116597153518,0.0420995244821,0.0692262392951,0.0546558190972,0.0692262392951,0.0293894980059,0.0880784281569,0.148602906238,0.14718449106,0.0453723719868,0.0364799780137,0.143740136859,0.0335358779938,0.179727178534,0.104190444502,0.0732586401789,0.0,0.0440392140784,0.0667245056621,0.0394544471286,0.106332536461,0.0526589109777,0.0547199670205,0.0335358779938,0.264932083908,0.0300149322

## Task 6) Train some classifiers 

Use the `LabeledPoint` objects to train a classifier, specifically the *LogisticRegression*, *Naive Bayes*, and *Support Vector Machine*. Calculate the accuracy of the model on the training set (again, follow this example [http://spark.apache.org/docs/2.1.0/ml-classification-regression.html#logistic-regression](http://spark.apache.org/docs/2.0.0/ml-classification-regression.html#logistic-regression) and here is the documentation for the classifiers [LogisticRegressionWithLBFGS](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.LogisticRegressionWithLBFGS), [NaiveBayes](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.NaiveBayes), [SVMWithSGD](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.SVMWithSGD).  (10%) 

In [61]:
from pyspark.mllib.classification import (NaiveBayes, LogisticRegressionWithLBFGS, SVMWithSGD) 
import numpy

# train the model with an (f,[(w,c), ...]) RDD. This is practical as we can reuse the function for TF.IDF
def trainModel(lpRDD):
    """ Train 3 classifier models on the given RDD with LabeledPoint objects. A list of trained model is returned. """
    lpRDD.persist(StorageLevel.MEMORY_ONLY) # not really needed as the Spark implementations ensure caching themselves. 
                    # Other implementations might not, however. 
    # Train a classifier model.
    print('Starting to train the model') # give some immediate feedback
    model1 = LogisticRegressionWithLBFGS.train(lpRDD) # this is the best model
    print('Trained LR (model1)')
    #print('type(model1)')
    model2 = NaiveBayes.train(lpRDD) # doesn't work well
    print('Trained NB (model2)')
    #print(type(model2))
    model3 = SVMWithSGD.train(lpRDD, convergenceTol=0.0001) # or this ...
    print('Trained SVM (model3)')
    return [model1,model2,model3]

def testModel(model, lpRDD):
    """ Tests the classification accuracy of the given model on the given RDD with LabeledPoint objects. """
    lpRDD.persist(StorageLevel.MEMORY_ONLY)
    # Make prediction and evaluate training set accuracy.
    # Get the prediction and the ground truth label
    predictionAndLabel = lpRDD.map(lambda p: (model.predict(p.features), p.label)) # get the prediction and ground truth (label) for each item.
    correct = predictionAndLabel.filter(lambda xv: xv[0] == xv[1]).count() # count the correct predictions 
    spam_count = predictionAndLabel.filter(lambda xv: xv[0] == 1).count() # count the number of spam predictions 
    notspam_count = predictionAndLabel.filter(lambda xv: xv[0] == 0).count() # count the number of spam predictions 
    print("spam count =", spam_count, "\nnotspam count = ", notspam_count)
    # >>>    accuracy = ... # and calculate the accuracy 
    accuracy = correct / predictionAndLabel.count() # and calculate the accuracy 
    print('Accuracy {:.1%} (data items: {}, correct: {})'.format(accuracy,lpRDD.count(), correct)) # report to console
    print("predictionAndLabel count", predictionAndLabel.count(), "lpRDD count", lpRDD.count()) # Just checking these values are the same
    return accuracy # and return the value  

models = trainModel(trainLpRDD) # just for testing
testModel(models[2], trainLpRDD) # just for testing

Starting to train the model
Trained LR (model1)
Trained NB (model2)
Trained SVM (model3)
spam count = 0 
notspam count =  2602
Accuracy 83.4% (data items: 2602, correct: 2170)
predictionAndLabel count 2602 lpRDD count 2602


0.8339738662567256

## Task 7) Automate training and testing

We automate now the whole process from reading the files, through preprocessing, and training up to evaluating the models. In the end we have a single function that takes all the parameters we are interested in and produces trained models and an evaluation. (5%) 

In [62]:
# this function combines tasks f) and g)
# this method should take RDDs with (f,[(w,c), ...])
def trainTestModel(trainRDD,testRDD):
    """ Trains 3 models and tests them on training and test data. Returns a matrix the training and testing (rows) accuracy values for all models (columns). """
    # >>> models = ... # train models on the training set
    models = trainModel(trainRDD) # train models on the training set
    results = [[],[]] # matrix for 2 models (training/test) vs n models (currently 3)
    for mdl in models:
        print('Training')
        # >>>     results[0].append(...) # test the model on the training set
        results[0].append((testModel(mdl, trainRDD) * 100)) # test the model on the training set 
        print('Testing')
        # >>>     results[1].append(...) # test the model on the test set
        results[1].append((testModel(mdl, testRDD) * 100)) # test the model on the test set
    return results

def trainTestFolder(folder,N):
    """ Reads data from a folder, preproceses the data, and trains and evaluates models on it. """
    print('Start loading and preprocessing') 
    train_test_LpRDD = loadAndPreprocess(folder,N) # create the RDDs
    print('Finished loading and preprocessing')
    (trainLpRDD,testLpRDD) = train_test_LpRDD # unpack the RDDs 
    return trainTestModel(trainLpRDD,testLpRDD) # train and test

trainTestFolder('lemm',1000) 

Start loading and preprocessing
Finished loading and preprocessing
Starting to train the model
Trained LR (model1)
Trained NB (model2)
Trained SVM (model3)
Training
spam count = 432 
notspam count =  2170
Accuracy 100.0% (data items: 2602, correct: 2602)
predictionAndLabel count 2602 lpRDD count 2602
Testing
spam count = 50 
notspam count =  241
Accuracy 97.6% (data items: 291, correct: 284)
predictionAndLabel count 291 lpRDD count 291
Training
spam count = 284 
notspam count =  2318
Accuracy 94.3% (data items: 2602, correct: 2454)
predictionAndLabel count 2602 lpRDD count 2602
Testing
spam count = 26 
notspam count =  265
Accuracy 92.1% (data items: 291, correct: 268)
predictionAndLabel count 291 lpRDD count 291
Training
spam count = 0 
notspam count =  2602
Accuracy 83.4% (data items: 2602, correct: 2170)
predictionAndLabel count 2602 lpRDD count 2602
Testing
spam count = 0 
notspam count =  291
Accuracy 83.2% (data items: 291, correct: 242)
predictionAndLabel count 291 lpRDD count 2

[[100.0, 94.31206764027671, 83.39738662567257],
 [97.59450171821305, 92.09621993127148, 83.16151202749141]]

## Task 8) Run experiments 

We have now a single function that allows us to vary the vector size easily. Test vector sizes 5, 50, 500, 5000, 50000 and examine the effect on the classification accuracy in Experiment 1.

Use the function from Task 7) to test different data types. The dataset has raw text in folder `bare`, lemmatised text in  `lemm` (similar to stemming, reduces to basic word forms), `stop` (with stopwords removed), and `lemm_stop` (lemmatised and stopwords removed). Test how the classification accuracy differs for these four data types in Experiment 2. Collect the results in a data structure that can be saved for later saving and analyis.

Comment on the results in a few sentences, considering the differences in performance between the different conditions as well as train an test values. 15%

In [67]:
from pyspark.sql import DataFrame
from pyspark.sql import Row

folder = 'bare'
# N = numpy.array([3,30,300,3000,30000])
N = numpy.array([5,50,500,5000,50000])
# N = numpy.array([5, 6])
print('\nEXPERIMENT 1: Testing different vector sizes')
models = ['LR', 'NB', 'SVM']
results = []

mergedf_RDD = sc.emptyRDD() # Initialise RDD into which rows generated from for loop will be added

# for n in N:
#     print('N = {}'.format(n))
#     training_test_accuracy = trainTestFolder(folder,n)
#     training_test = list(zip(*training_test_accuracy)) # zip corresponding test and training values together
#     models_training_test = list(zip(models, training_test)) # zip model to corresponding test/training accuracy tuple
#     # Convert list to RDD
#     accuracy_rdd = sc.parallelize(models_training_test)
#     # Map values of RDD to a data frame Row object
#     accuracy_rdd_rows = accuracy_rdd.map(lambda mod_tr_te, n=int(n): Row(model = mod_tr_te[0], training_acc = mod_tr_te[1][0], test_acc = mod_tr_te[1][1], vector_dimensions = n, folder = 'bare'))

#     # Merge each generate Row into final RDD output
#     mergedf_RDD = mergedf_RDD.union(accuracy_rdd_rows)
    
# mergedf_RDD.toDF().show()

mergedf_folders_RDD = sc.emptyRDD() # Initialise RDD into which rows generated from for loop will be added     
n = 7000
typeFolders = ['bare','stop','lemm','lemm_stop']
print('EXPERIMENT 2: Testing different data types')
for folder in typeFolders:
    print('Path = {}'.format(folder))
    training_test_accuracy = trainTestFolder(folder,n)
    training_test = list(zip(*training_test_accuracy)) # zip corresponding test and training values together
    models_training_test = list(zip(models, training_test)) # zip model to corresponding test/training accuracy tuple
    # Convert list to RDD
    accuracy_rdd = sc.parallelize(models_training_test)
    # Map values of RDD to a data frame Row object
    accuracy_rdd_rows = accuracy_rdd.map(lambda mod_tr_te, folder=folder: Row(model = mod_tr_te[0], training_acc = mod_tr_te[1][0], test_acc = mod_tr_te[1][1], vector_dimensions = 3000, folder = folder))
    # Merge each generate Row into final RDD output
    mergedf_folders_RDD = mergedf_folders_RDD.union(accuracy_rdd_rows)

mergedf_folders_RDD.toDF().show()



# Add comments on the performance in a cell below. 


EXPERIMENT 1: Testing different vector sizes
EXPERIMENT 2: Testing different data types
Path = bare
Start loading and preprocessing
Finished loading and preprocessing
Starting to train the model
Trained LR (model1)
Trained NB (model2)
Trained SVM (model3)
Training
spam count = 432 
notspam count =  2170
Accuracy 100.0% (data items: 2602, correct: 2602)
predictionAndLabel count 2602 lpRDD count 2602
Testing
spam count = 47 
notspam count =  244
Accuracy 97.9% (data items: 291, correct: 285)
predictionAndLabel count 291 lpRDD count 291
Training
spam count = 342 
notspam count =  2260
Accuracy 96.5% (data items: 2602, correct: 2512)
predictionAndLabel count 2602 lpRDD count 2602
Testing
spam count = 31 
notspam count =  260
Accuracy 93.8% (data items: 291, correct: 273)
predictionAndLabel count 291 lpRDD count 291
Training
spam count = 0 
notspam count =  2602
Accuracy 83.4% (data items: 2602, correct: 2170)
predictionAndLabel count 2602 lpRDD count 2602
Testing
spam count = 0 
notspam c

In [ ]:
# Comment on the results in a few sentences, considering the differences in performance between the different conditions 
# as well as train an test values. 15%
# In experiment 1 we explore the effect of increasing the dimension size of the vector from 5 to 50,000. 
# We see that overall the best performing model is LR followed by NB followed by SVM. 
# As vector size was increased from 5 to 5000 an increase in the training and test accuracy values were observed for both the LR and NB models. 
# This demonstrated that increasing the size of the vector that represents each document helps to improve the capacity of these models to differentiate spam from other emails.
# These findings indicate that as the vector size is increased the amount of information/entropy that is encoded increases. 
# This is because as the vector size is increased there are less occurances when different terms are assigned to the same vector value as defined by hash()% vector_size i.e. less collisions.
# This helps to minimise misclassifications of different terms.
# The improvement in test accuracy alongside improvements in training accuracy for 5-5000 vector size range shows that the optimised LR and NB models generalise well to unseen data.
# Increasing the vector size from 5000 to 50000 does not affect the performance of the LR model (no indication that data is being overfitted) but we do observe a decrease in the accuracy of the NB model both for the training and test data.
# This most likely occurs because the sample size of the training set (2602 documents) is not large enough to accurately model the prior probability distribution of a 50,000 dimensional vector space for the document population.
# The SVM model showed no improvement in accuracy across the vector dimensional space that was explored. 
# This is because the SVM model employed here utlises a linear kernal and so irrespective of the increase in the vector dimension it is unable to model the non-linear hyperplane that would allow the best separation of spam versus other email vectors.
# 
# In experiment 2 we explore the effect of using different text content to classify spam from other emails. Similar to experiment 1 LR demonstrated the highest accuracy followed by NB followed by SVM across the 4 text types (raw text vs raw text minus stop words vs lemmatised text vs lemmatised text with stop words removed).
# The function of lemmatising words and removing stop words is to distill the text into functional units that can be uniformly counted across different documents. 
# Removing stop words or lemmatising the text did not improve predictive accuracy of the LR model but combining the removal of stop words with lemmatising the text did result in a small increase in accuracy in test data.
# In contrast reducing information in the text by removing stop words and lemmatising decreased the predictive accuracy of the NB model. 
# Interestingly it appears that stop words and lemmatising make separate and distinct contributions to the NB model whereby combining the removal of stop words with lemmatising the text resulted in a lower predictive accuracy compared with removing stop words or lemmatising on its own.
# This possibly suggests that the added variance in the data by keeping stop words and not lemmatising helps to ensure the prior probability distribution of a document's classification (i.e. spam/other) more closely represents that of the document population.
# This ensures that the NB model generates more accurate posterior probability estimates (better prediction accuracy) on the test data.
# Similar to experiment 1 the SVM model showed no marked change in performance across the different text types. 
# As in experiment 1 the SVM model employed in this experiemnts uses a linear kernal which is unable to model the non-linear hyperplane that best separates the data into spam and other emails.
# Hence using different text types did not change the predictive accuracy of the SVM model.
# Further investigation revealed that the SVM model did not predict any of the emails were spam and simply labelled all emails as not spam. See spam and not spam count output in task 6.
# Therefore the value of 83% identified by the SVM model relates to the proportion of emails that are not spam in the test set. 
# This aligns with the known ratio of not spam to spam emails in the ling spam data set which is reported to equate to 83.4% not spam : 16.6% spam (Androutsopoulos et al. 2000).

# Reference 
# Androutsopoulos, I., Koutsias, J., Chandrinos K.V., Paliouras G., Spyropoulos C.D., An Evaluation of Naive Bayesian Anti-Spam Filtering. Proceedings of the workshop on Machine Learning in the New Information Age. (2000)

In [64]:
# For ease the summary tables for the analysis shown in Task 8 are shown below
mergedf_RDD.toDF().show()

mergedf_folders_RDD.toDF().show()


+------+-----+-----------------+-----------------+-----------------+
|folder|model|         test_acc|     training_acc|vector_dimensions|
+------+-----+-----------------+-----------------+-----------------+
|  bare|   LR|83.16151202749141|83.39738662567257|                5|
|  bare|   NB|83.16151202749141|83.39738662567257|                5|
|  bare|  SVM|83.16151202749141|83.39738662567257|                5|
|  bare|   LR|87.97250859106529|93.92774788624135|               50|
|  bare|   NB|83.16151202749141|83.39738662567257|               50|
|  bare|  SVM|83.16151202749141|83.39738662567257|               50|
|  bare|   LR|95.53264604810997|            100.0|              500|
|  bare|   NB|84.87972508591065|88.66256725595696|              500|
|  bare|  SVM|83.16151202749141|83.39738662567257|              500|
|  bare|   LR| 97.9381443298969|            100.0|             5000|
|  bare|   NB|94.84536082474226|97.04073789392775|             5000|
|  bare|  SVM|83.16151202749141|83